In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
from IMM import IMM
from Junction_Condtion import JC
from MMJ import MMJ
# import sys
# sys.path.append('..')

In [2]:
# def point_matching(curr_loc, curr_edge):
#     # matched position to the current edge 
#     # input need to be panda series
#     # curr loc need attribute geometry point
#     # curr_edge need attribute 'geometry' lines
#     # output a point that matched to the current edge
#     crs_utm = curr_loc.crs
#     dist = curr_edge['geometry'].project(curr_loc['geometry']).iloc[0]
#     matched_point = list(curr_edge['geometry'].interpolate(dist).coords)
#     matched_point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([matched_point[0][0]], [matched_point[0][1]]), crs= crs_utm)
#     return matched_point

In [3]:
def AHPMM(trajector_data, nodes_data, edges_data):
    matched_links = []
    matched_links_no_duplicate = []
    # matched_points = []
    iter, previous_edge = IMM(trajectory_data, edges_data, iter=0)
    previous_edge['method'] = ['IMM']
    matched_links.append(previous_edge)
    matched_links_no_duplicate.append(previous_edge)
    while iter < len(trajectory_data):
        current_position = trajectory_data.iloc[[iter]]
        previous_position = trajectory_data.iloc[[iter-1]]
        # print(current_position)
        if JC(nodes_data, edges_data, current_position, previous_position, previous_edge):
            previous_edge['method'] = ['JC']
            matched_links.append(previous_edge)
            iter = iter + 1
        else:
            temp = previous_edge
            iter, previous_edge = MMJ(trajectory_data, nodes_data, edges_data, current_position, previous_edge, iter)
            previous_edge['method'] = ['SMM']
            matched_links.append(previous_edge)
            if (previous_edge.index[0] != temp.index[0]) and (previous_edge.iloc[0].method == 'SMM'):
                matched_links_no_duplicate.append(previous_edge)

        # matched_point = point_matching(current_position, previous_edge.iloc[0])
        # # find longitude and latitude for last matched data
        # matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
        # matched_points.append(matched_point)
    return matched_links, matched_links_no_duplicate #, matched_points

In [4]:
import sys
sys.path.append('..')
import preprocessing.downloader as dl
# id = dl.fetch_ids_on_envirocar(num_trajectories=3)[2]
id = "64c35f12c7e42807e7b8fc4e"
gdf = dl.download_from_envirocar(id, use_cache=False, threshold=10)
# print(dl.fetch_ids_on_envirocar(num_trajectories=3))
trajectory_data, nodes_data, edges_data = dl.to_fuzzy_AHP_input(gdf)

In [5]:
# trajectory_data = pd.read_pickle('gdf_utm.pkl')
# nodes_data = pd.read_pickle('nodes_utm.pkl')
# edges_data = pd.read_pickle('edges_utm.pkl')
# initial_previous_position = pd.read_pickle('gdf_utm.pkl').iloc[iter].to_frame()
# matched_links, matched_points = AHPMM(trajectory_data, nodes_data, edges_data)
matched_links, matched_links_no_duplicate = AHPMM(trajectory_data, nodes_data, edges_data)

In [6]:
matched_edge_df = pd.concat(matched_links)
# matched_points_df = pd.concat(matched_points)

In [7]:
# visualization
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = trajectory_data['geometry'][150:180].to_frame()
point_locs.plot(ax = ax, color="Red")


# this plot all the road system 
edges_data.plot(ax=ax)

# this plot the selected edge at time point 


matched_edge_df.plot(ax = ax, color = "Yellow")


# matched_points_df.plot(ax = ax, color = "Red")

<Axes: >

In [8]:
pd.concat(matched_links_no_duplicate).tail(20)

,,,osmid,oneway,lanes,ref,name,highway,maxspeed,reversed,length,width,...,junction,lon_lat,str_id,perp_dist,lon_lat_pair,edge_heading,heading_error,method,prev_end_node,connectivity
u,v,key,,,,,,,,,,,,,,,,,,,,,
3359581623,3359581636,0,329139427,False,NaN,NaN,NaN,footway,NaN,False,43.357,NaN,...,NaN,"LINESTRING (9.17040 48.78247, 9.17049 48.78279...",329139427,28.190859,"[(9.1704027, 48.7824693), (9.1704899, 48.78279...",11.627818,25.798632,SMM,3.359582e+09,1.0
3359581636,281410022,0,"[25799257, 329139423]",True,3,NaN,Herdweg,tertiary,"[40, 50]",False,52.892,NaN,...,NaN,"LINESTRING (9.17052 48.78285, 9.17064 48.78269...","25799257,329139423",10.334309,"[(9.1705214, 48.7828505), (9.1706359, 48.78268...",163.559072,167.703149,SMM,3.359582e+09,1.0
18240880,271727242,0,"[24817067, 581189438]",False,"[3, 2]",NaN,Herdweg,tertiary,40,False,41.931,NaN,...,NaN,"LINESTRING (9.17050 48.78293, 9.17032 48.78305...","24817067,581189438",4.660178,"[(9.1704976, 48.7829256), (9.1703233, 48.78305...",318.979982,1.397945,SMM,2.814100e+08,0.0
271727242,3359527265,0,24817067,False,2,NaN,Herdweg,tertiary,40,False,7.115,NaN,...,NaN,"LINESTRING (9.17012 48.78321, 9.17006 48.78326)",24817067,24.286230,"[(9.1701227, 48.7832104), (9.1700578, 48.783258)]",317.980668,1.390435,SMM,2.717272e+08,1.0
3359527265,2847671068,0,"[581133379, 24817067]",False,"[3, 2]",NaN,Herdweg,tertiary,40,False,56.209,NaN,...,NaN,"LINESTRING (9.17006 48.78326, 9.16977 48.78348...","581133379,24817067",19.671107,"[(9.1700578, 48.783258), (9.1697685, 48.783477...",319.458513,0.635950,JC,3.359527e+09,1.0
2847671068,151003073,0,581133379,False,3,NaN,Herdweg,tertiary,40,False,42.504,NaN,...,NaN,"LINESTRING (9.16956 48.78364, 9.16929 48.78385...",581133379,20.750870,"[(9.16956, 48.7836426), (9.1692866, 48.7838522...",320.196062,3.621606,SMM,2.847671e+09,1.0
151003073,277415336,0,"[1101069146, 120879116, 581134341]",False,"[3, 2]",NaN,Herdweg,tertiary,40,True,134.801,NaN,...,NaN,"LINESTRING (9.16919 48.78394, 9.16909 48.78403...","1101069146,120879116,581134341",41.030451,"[(9.1691894, 48.7839365), (9.1690911, 48.78403...",315.993713,3.607788,JC,1.510031e+08,1.0
277415336,151003073,0,"[1101069146, 120879116, 581134341]",False,"[3, 2]",NaN,Herdweg,tertiary,40,False,134.801,NaN,...,NaN,"LINESTRING (9.16792 48.78481, 9.16811 48.78471...","1101069146,120879116,581134341",3.077942,"[(9.1679189, 48.7848057), (9.1681069, 48.78471...",135.992757,168.738711,SMM,2.774153e+08,1.0
151003073,277415336,0,"[1101069146, 120879116, 581134341]",False,"[3, 2]",NaN,Herdweg,tertiary,40,True,134.801,NaN,...,NaN,"LINESTRING (9.16919 48.78394, 9.16909 48.78403...","1101069146,120879116,581134341",3.941798,"[(9.1691894, 48.7839365), (9.1690911, 48.78403...",315.993713,9.834014,SMM,1.510031e+08,1.0
